# 1. Importing Libraries and Data Set

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
### Define Path

path = r'/Users/aislingm/Documents/Achievement 6'

### Import Boat sales data

df = pd.read_csv(os.path.join(path, 'boat_data.csv'))

# 2. Data Checks

In [3]:
df.head()

,Price,Boat Type,Manufacturer,Type,Year Built,Length,Width,Material,Location,Number of views last 7 days
0,CHF 3337,Motor Yacht,Rigiflex power boats,new boat from stock,2017,4.00,1.90,NaN,Switzerland Â» Lake Geneva Â» VÃ©senaz,226
1,EUR 3490,Center console boat,Terhi power boats,new boat from stock,2020,4.00,1.50,Thermoplastic,Germany Â» BÃ¶nningstedt,75
2,CHF 3770,Sport Boat,Marine power boats,new boat from stock,0,3.69,1.42,Aluminium,Switzerland Â» Lake of Zurich Â» StÃ¤fa ZH,124
3,DKK 25900,Sport Boat,Pioner power boats,new boat from stock,2020,3.00,1.00,NaN,Denmark Â» Svendborg,64
4,EUR 3399,Fishing Boat,Linder power boats,new boat from stock,2019,3.55,1.46,Aluminium,Germany Â» Bayern Â» MÃ¼nchen,58


In [4]:
df.shape

(9888, 10)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9888 entries, 0 to 9887
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Price                        9888 non-null   object 
 1   Boat Type                    9888 non-null   object 
 2   Manufacturer                 8550 non-null   object 
 3   Type                         9882 non-null   object 
 4   Year Built                   9888 non-null   int64  
 5   Length                       9879 non-null   float64
 6   Width                        9832 non-null   float64
 7   Material                     8139 non-null   object 
 8   Location                     9852 non-null   object 
 9   Number of views last 7 days  9888 non-null   int64  
dtypes: float64(2), int64(2), object(6)
memory usage: 772.6+ KB


# 3. Data cleaning and wrangling

#### A first glance at our data tells us that there entries in the year built column with a value of 0. The location and price columns will need some manipulation such as removing what is written after the country and having one unique currency in order to understand how the price will play a role in our study. The material column also has missing values shown as NaN. First let's take a look at the missing data in order to determine how to proceed with such values. Let's first take a look at missing values.

In [6]:
# % value of missing entries in each column
(df.isnull().sum()/len(df) * 100)

Price                           0.000000
Boat Type                       0.000000
Manufacturer                   13.531553
Type                            0.060680
Year Built                      0.000000
Length                          0.091019
Width                           0.566343
Material                       17.688107
Location                        0.364078
Number of views last 7 days     0.000000
dtype: float64

#### As the % of missing values for year built is less than 1 we will drop those values

In [7]:
# Drop rows where 'Year Built' is 0
df = df[df['Year Built']!=0]

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9337 entries, 0 to 9887
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Price                        9337 non-null   object 
 1   Boat Type                    9337 non-null   object 
 2   Manufacturer                 8051 non-null   object 
 3   Type                         9332 non-null   object 
 4   Year Built                   9337 non-null   int64  
 5   Length                       9328 non-null   float64
 6   Width                        9282 non-null   float64
 7   Material                     7675 non-null   object 
 8   Location                     9301 non-null   object 
 9   Number of views last 7 days  9337 non-null   int64  
dtypes: float64(2), int64(2), object(6)
memory usage: 802.4+ KB


#### Material and Manufacturer have a high % of missing values so replacing them with the mean or dropping them could impact our analysis. Let's take a further look at these columns.

In [9]:
# Explore rows where 'Material' is missing
df[df['Material'].isnull()]

,Price,Boat Type,Manufacturer,Type,Year Built,Length,Width,Material,Location,Number of views last 7 days
0,CHF 3337,Motor Yacht,Rigiflex power boats,new boat from stock,2017,4.00,1.90,NaN,Switzerland Â» Lake Geneva Â» VÃ©senaz,226
3,DKK 25900,Sport Boat,Pioner power boats,new boat from stock,2020,3.00,1.00,NaN,Denmark Â» Svendborg,64
8,EUR 3333,Fishing Boat,Crescent power boats,new boat from stock,2019,3.64,1.37,NaN,Germany Â» Bayern Â» Boote+service Oberbayern,45
9,EUR 3300,Pontoon Boat,Whaly power boats,new boat from stock,2018,4.35,1.73,NaN,Italy Â» Dormelletto,180
17,DKK 30000,Sport Boat,NaN,Used boat,1985,4.00,1.00,NaN,Denmark Â» Svendborg,124
...,...,...,...,...,...,...,...,...,...,...
9866,DKK 36500,Sport Boat,NaN,Used boat,1985,4.00,1.00,NaN,Denmark Â» Svendborg,96
9867,DKK 36500,Fishing Boat,Hunter power boats,Used boat,2000,5.00,1.00,NaN,Denmark Â» Svendborg,87
9873,EUR 4799,Working Boat,NaN,"new boat from stock,Electric",2019,3.64,1.37,NaN,Germany Â» Bayern Â» Boote Jochum,41
9883,CHF 4900,Sport Boat,Sea Ray power boats,"Used boat,Unleaded",1987,6.30,2.44,NaN,Switzerland Â» Lago Maggiore Â» Riazzino,1116


In [10]:
# Explore rows where 'Manufacturer' is missing
df[df['Manufacturer'].isnull()]

,Price,Boat Type,Manufacturer,Type,Year Built,Length,Width,Material,Location,Number of views last 7 days
6,CHF 3600,Catamaran,NaN,"Used boat,Unleaded",1999,6.20,2.38,Aluminium,Switzerland Â» Neuenburgersee Â» Yvonand,474
17,DKK 30000,Sport Boat,NaN,Used boat,1985,4.00,1.00,NaN,Denmark Â» Svendborg,124
19,EUR 4000,Cabin Boat,NaN,"Used boat,Unleaded",2011,6.37,2.31,GRP,Germany Â» Bayern Â» Forchheim/Ofr.,330
24,EUR 3900,Cabin Boat,NaN,Used boat,1985,6.53,2.47,NaN,"France Â» Marseille, France",383
28,CHF 4000,Fishing Boat,NaN,"Used boat,Unleaded",1994,5.60,1.58,GRP,Switzerland Â» Lake of Zurich Â» Altendorf SZ,641
...,...,...,...,...,...,...,...,...,...,...
9862,EUR 4999,Pilothouse,NaN,"Used boat,Diesel",1955,9.00,3.00,GRP,Germany Â» Schleswig-Holstein Â» Burg/Dithmars...,173
9863,EUR 4990,Cabin Boat,NaN,Unleaded,1987,6.50,2.50,GRP,Germany Â» Verkaufshafen Brandenburg-Plaue,190
9866,DKK 36500,Sport Boat,NaN,Used boat,1985,4.00,1.00,NaN,Denmark Â» Svendborg,96
9873,EUR 4799,Working Boat,NaN,"new boat from stock,Electric",2019,3.64,1.37,NaN,Germany Â» Bayern Â» Boote Jochum,41


#### In both cases there are to many rows with missing data. Using the most common material type (GRP) for imputation involves numerous assumptions, and it has the potential to bias the final results in favor of the predominant material. The same can be stated about 'Manufacturer' as 'Material' column. Additionally, seing the high % of missing values for these columns may spike the stakeholder's interest in future improvements to enhance the listing survey therefor the decision has been made to retain the NaN values in these instances but we will replace them with none

In [11]:
# Replace NaNs with 'none' in Manufacturer and Material columns, drop the rest
df[['Manufacturer', 'Material']] = df[['Manufacturer', 'Material']].fillna(value='none')
df = df.dropna()

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9241 entries, 0 to 9887
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Price                        9241 non-null   object 
 1   Boat Type                    9241 non-null   object 
 2   Manufacturer                 9241 non-null   object 
 3   Type                         9241 non-null   object 
 4   Year Built                   9241 non-null   int64  
 5   Length                       9241 non-null   float64
 6   Width                        9241 non-null   float64
 7   Material                     9241 non-null   object 
 8   Location                     9241 non-null   object 
 9   Number of views last 7 days  9241 non-null   int64  
dtypes: float64(2), int64(2), object(6)
memory usage: 794.1+ KB


#### The location data has information about what seems to be the lakes and it makes for a complex untidy column. So extracting the country from the location column and dropping the location column I believe is best suited for our analysis

In [13]:
# Extracting country from 'Location'
df['Country'] = df['Location'].str.split('Â»', expand=True)[0].str.strip()

# Dropping 'Location'
df = df.drop(columns='Location')

In [14]:
df.head()

,Price,Boat Type,Manufacturer,Type,Year Built,Length,Width,Material,Number of views last 7 days,Country
0,CHF 3337,Motor Yacht,Rigiflex power boats,new boat from stock,2017,4.00,1.90,none,226,Switzerland
1,EUR 3490,Center console boat,Terhi power boats,new boat from stock,2020,4.00,1.50,Thermoplastic,75,Germany
3,DKK 25900,Sport Boat,Pioner power boats,new boat from stock,2020,3.00,1.00,none,64,Denmark
4,EUR 3399,Fishing Boat,Linder power boats,new boat from stock,2019,3.55,1.46,Aluminium,58,Germany
6,CHF 3600,Catamaran,none,"Used boat,Unleaded",1999,6.20,2.38,Aluminium,474,Switzerland


#### The price column includes the cost and the currency. We will need to separate both values and convert all of the prices into the same currency. I will choose Euro for easier undertanding of my analysis.

In [15]:
# Extract type of currency and amount from Price column
df[['Currency', 'Price']] = df['Price'].str.split(' ', 1, expand=True)

# Changing data type of 'Price' column to numeric
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

# Display the updated DataFrame
print(df)


      Price            Boat Type           Manufacturer  \
0      3337          Motor Yacht   Rigiflex power boats   
1      3490  Center console boat      Terhi power boats   
3     25900           Sport Boat     Pioner power boats   
4      3399         Fishing Boat     Linder power boats   
6      3600            Catamaran                   none   
...     ...                  ...                    ...   
9882   4950         Fishing Boat  Staempfli power boats   
9883   4900           Sport Boat    Sea Ray power boats   
9885   4499           Sport Boat  BlueCraft power boats   
9886   4300         Pontoon Boat      Whaly power boats   
9887   3780         Fishing Boat    DarekCo power boats   

                              Type  Year Built  Length  Width       Material  \
0              new boat from stock        2017    4.00   1.90           none   
1              new boat from stock        2020    4.00   1.50  Thermoplastic   
3              new boat from stock        2020    3

/var/folders/m4/vmdq9y9n2fndv5637y77b2nh0000gn/T/ipykernel_9599/1781323358.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['Currency', 'Price']] = df['Price'].str.split(' ', 1, expand=True)


In [16]:
# Check unique currencies
unique_currencies = df['Currency'].unique()

# Display the unique currencies
print(unique_currencies)

['CHF' 'EUR' 'DKK' 'Â£']


#### I will convert all costs to Euro and then drop the currency column and rename the price column to include the fact that the prices are in Euro. I will use the convertion amount from November 2023

In [17]:
# Define conversion factors
conversion_factors = {'CHF': 0.96, 'DKK': 7.46, 'Â£': 0.87}

# Apply the conversion to 'Price' column
df['Price'] = df.apply(lambda row: row['Price'] * conversion_factors[row['Currency']] if row['Currency'] in conversion_factors else row['Price'], axis=1)

# Update 'Currency' column to 'EUR' for all prices
df['Currency'] = 'EUR'

# Display the updated DataFrame
print(df)

          Price            Boat Type           Manufacturer  \
0       3203.52          Motor Yacht   Rigiflex power boats   
1       3490.00  Center console boat      Terhi power boats   
3     193214.00           Sport Boat     Pioner power boats   
4       3399.00         Fishing Boat     Linder power boats   
6       3456.00            Catamaran                   none   
...         ...                  ...                    ...   
9882    4752.00         Fishing Boat  Staempfli power boats   
9883    4704.00           Sport Boat    Sea Ray power boats   
9885    4499.00           Sport Boat  BlueCraft power boats   
9886    4300.00         Pontoon Boat      Whaly power boats   
9887    3628.80         Fishing Boat    DarekCo power boats   

                              Type  Year Built  Length  Width       Material  \
0              new boat from stock        2017    4.00   1.90           none   
1              new boat from stock        2020    4.00   1.50  Thermoplastic   
3  

In [18]:
# Drop 'Currency' column
df = df.drop(columns='Currency')

# Rename 'Price' column to 'Price_Eur'
df = df.rename(columns={'Price': 'Price_Eur'})

# Display the updated DataFrame
print(df)

      Price_Eur            Boat Type           Manufacturer  \
0       3203.52          Motor Yacht   Rigiflex power boats   
1       3490.00  Center console boat      Terhi power boats   
3     193214.00           Sport Boat     Pioner power boats   
4       3399.00         Fishing Boat     Linder power boats   
6       3456.00            Catamaran                   none   
...         ...                  ...                    ...   
9882    4752.00         Fishing Boat  Staempfli power boats   
9883    4704.00           Sport Boat    Sea Ray power boats   
9885    4499.00           Sport Boat  BlueCraft power boats   
9886    4300.00         Pontoon Boat      Whaly power boats   
9887    3628.80         Fishing Boat    DarekCo power boats   

                              Type  Year Built  Length  Width       Material  \
0              new boat from stock        2017    4.00   1.90           none   
1              new boat from stock        2020    4.00   1.50  Thermoplastic   
3  

#### I want to further explore the 'Type' column and determine if it makes sense to add a new column labeled 'Fuel Type'. To do this I will analyze the unique values in the 'Type' column and identify patterns. The reason for this is because it seems like some rows contain information about the fuel type, while others do not.

In [19]:
# Examine Unique values

print(df['Type'].unique())

['new boat from stock' 'Used boat,Unleaded' 'Used boat,Electric'
 'Used boat' 'Used boat,Diesel' 'new boat from stock,Unleaded' 'Unleaded'
 'new boat on order,Unleaded' 'new boat from stock,Gas'
 'new boat on order' 'Diesel' 'Display Model,Unleaded' 'Display Model'
 'Used boat,Gas' 'new boat from stock,Diesel'
 'new boat from stock,Electric' 'new boat on order,Diesel'
 'new boat from stock,Hybrid' 'Display Model,Diesel' 'Used boat,Hybrid'
 'Display Model,Gas' 'Display Model,Electric' 'Electric'
 'Used boat,Propane']


In [20]:
# Create new columns 'Boat Condition' and 'Fuel Type'

df['Boat Condition'] = df['Type'].str.replace(r',.*$', '').str.strip()
df['Fuel Type'] = df['Type'].str.extract(r',\s*(.*)$', expand=False).str.strip()

# Display DataFrame with the new columns

print(df[['Type', 'Boat Condition', 'Fuel Type']])


                              Type       Boat Condition Fuel Type
0              new boat from stock  new boat from stock       NaN
1              new boat from stock  new boat from stock       NaN
3              new boat from stock  new boat from stock       NaN
4              new boat from stock  new boat from stock       NaN
6               Used boat,Unleaded            Used boat  Unleaded
...                            ...                  ...       ...
9882            Used boat,Unleaded            Used boat  Unleaded
9883            Used boat,Unleaded            Used boat  Unleaded
9885  new boat from stock,Unleaded  new boat from stock  Unleaded
9886           new boat from stock  new boat from stock       NaN
9887           new boat from stock  new boat from stock       NaN

[9241 rows x 3 columns]


/var/folders/m4/vmdq9y9n2fndv5637y77b2nh0000gn/T/ipykernel_9599/2195907131.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Boat Condition'] = df['Type'].str.replace(r',.*$', '').str.strip()


In [21]:
print(df)

      Price_Eur            Boat Type           Manufacturer  \
0       3203.52          Motor Yacht   Rigiflex power boats   
1       3490.00  Center console boat      Terhi power boats   
3     193214.00           Sport Boat     Pioner power boats   
4       3399.00         Fishing Boat     Linder power boats   
6       3456.00            Catamaran                   none   
...         ...                  ...                    ...   
9882    4752.00         Fishing Boat  Staempfli power boats   
9883    4704.00           Sport Boat    Sea Ray power boats   
9885    4499.00           Sport Boat  BlueCraft power boats   
9886    4300.00         Pontoon Boat      Whaly power boats   
9887    3628.80         Fishing Boat    DarekCo power boats   

                              Type  Year Built  Length  Width       Material  \
0              new boat from stock        2017    4.00   1.90           none   
1              new boat from stock        2020    4.00   1.50  Thermoplastic   
3  

In [22]:
# Count the number of missing values in the 'Fuel Type' column
missing_fuel_type = df['Fuel Type'].isnull().sum()

# Display the count of missing values
print(f"Number of missing values in 'Fuel Type': {missing_fuel_type}")


Number of missing values in 'Fuel Type': 2106


In [23]:
# Count the number of missing values in the 'Boat Condition' column
missing_boat_condition = df['Boat Condition'].isnull().sum()

# Display the count of missing values
print(f"Number of missing values in 'Boat Condition': {missing_boat_condition}")


Number of missing values in 'Boat Condition': 0


#### I believe that despite the high number of missing values in the Fuel Type column it may be interesting for our analysis to keep the new 2 columns 'Boat Condition' and 'Fuel Type' and to drop the originil Type column. I given the high number of missing values in the new Fuel Type column I do not believe that dropping the missing values would be wise nor replacing them with other than none.

In [24]:
# Replace NaN values in 'Fuel Type' with None
df['Fuel Type'].replace({pd.NA: None, np.nan: None}, inplace=True)

# Drop the 'Type' column
df.drop(columns=['Type'], inplace=True)

# Display the DataFrame
print(df)


      Price_Eur            Boat Type           Manufacturer  Year Built  \
0       3203.52          Motor Yacht   Rigiflex power boats        2017   
1       3490.00  Center console boat      Terhi power boats        2020   
3     193214.00           Sport Boat     Pioner power boats        2020   
4       3399.00         Fishing Boat     Linder power boats        2019   
6       3456.00            Catamaran                   none        1999   
...         ...                  ...                    ...         ...   
9882    4752.00         Fishing Boat  Staempfli power boats        1984   
9883    4704.00           Sport Boat    Sea Ray power boats        1987   
9885    4499.00           Sport Boat  BlueCraft power boats        2020   
9886    4300.00         Pontoon Boat      Whaly power boats        2018   
9887    3628.80         Fishing Boat    DarekCo power boats        2019   

      Length  Width       Material  Number of views last 7 days      Country  \
0       4.00   1.90

# 4. Exploratory Analysis

In [25]:
df.describe()

,Price_Eur,Year Built,Length,Width,Number of views last 7 days
count,9.241000e+03,9241.000000,9241.000000,9241.000000,9241.000000
mean,4.804265e+05,2004.931176,11.717052,3.552552,150.446813
std,2.678327e+06,16.387410,5.997217,1.211919,155.106784
min,3.203520e+03,1885.000000,1.040000,0.010000,13.000000
25%,4.499000e+04,1999.000000,7.560000,2.550000,70.000000
50%,9.800000e+04,2008.000000,10.500000,3.390000,108.000000
75%,2.600000e+05,2018.000000,14.000000,4.260000,172.000000
max,1.311328e+08,2021.000000,100.000000,25.160000,3263.000000


# 5. Save my new dataframe

In [26]:
df.to_csv(os.path.join(path, 'boat_data_cleaned.csv'), index=False)